In [1]:
import numpy as np
import pandas as pd
import re
import os


In [3]:
#these are all the data files we will be using, the .csvs are not in the repository for space reasons
#but the files are available on the google drive
price= pd.read_csv('NADAC_as_of_2017-06-07.csv',dtype={'NDC': str})
product= pd.read_csv('product.csv')
package=pd.read_csv('package.csv')
NADAC_Comparison = pd.read_csv('NADAC_Comparison.csv',dtype={'NDC':str})
price_series = pd.read_csv('NADAC__National_Average_Drug_Acquisition_Cost_.csv')



In [5]:
#here are various data cleaning operations we must do to analyze the data

#make percent change a float
NADAC_Comparison['Percent Change']=NADAC_Comparison['Percent Change'].apply(lambda x: x[:-1])
NADAC_Comparison['Percent Change']=NADAC_Comparison['Percent Change'].apply(pd.to_numeric)

#change the NDC so it matches the NDC in the other .csv
price['NDC10']=price['NDC'].apply(lambda x: x[0:5]+x[6:])  #de18.founded=de18.founded.apply(lambda x: x[8:])

#merge two csv's together
FDA = pd.merge(package,product,how='inner',on='PRODUCTNDC')

#remove the - from the NDC to match the other NDC
import re
def func(x):                                       # replace the - for ''in column:NDCPACKAGECODE for file FDA
    return re.sub('[-]',"", str(x))
FDA['NDC'] = FDA['NDCPACKAGECODE'].apply(func)

In [110]:
priceInfo = pd.merge(FDA,price,right_on='NDC10',left_on='NDC10')


,PRODUCTID_x,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,PRODUCTID_y,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_x,NDC10,NDC Description,NDC_y,NADAC_Per_Unit,Effective_Date,Pricing_Unit,Pharmacy_Type_Indicator,OTC,Explanation_Code,Classification_for_Rate_Setting,Corresponding_Generic_Drug_NADAC_Per_Unit,Corresponding_Generic_Drug_Effective_Date,As of Date
0,0009-3073_0f1820e6-28cb-4dd7-89ee-00583ebab35d,Sep-73,0009-3073-01,"1 VIAL, SINGLE-DOSE in 1 PACKAGE (0009-3073-01...",0009-3073_0f1820e6-28cb-4dd7-89ee-00583ebab35d,HUMAN PRESCRIPTION DRUG,Depo-Medrol,NaN,methylprednisolone acetate,"INJECTION, SUSPENSION",INTRA-ARTICULAR; INTRALESIONAL; INTRAMUSCULAR;...,19590528.0,NaN,NDA,NDA011757,Pharmacia and Upjohn Company LLC,METHYLPREDNISOLONE ACETATE,40,mg/mL,"Corticosteroid [EPC],Corticosteroid Hormone Re...",NaN,0009307301,0009307301,ZOLPIDEM TARTRATE 5 MG TABLET,00093007301,0.02878,05/17/2017,EA,C/I,N,1,G,NaN,NaN,06/07/2017


In [111]:
priceInfo = priceInfo.loc[:,['DOSAGEFORMNAME'] + ['ROUTENAME'] + ['STARTMARKETINGDATE'] + ['LABELERNAME']  + ['PHARM_CLASSES'] + ['DEASCHEDULE'] + ['NADAC_Per_Unit']]
def Dummify(column):
    column_values = pd.get_dummies(priceInfo[column], prefix=column, prefix_sep='__')
    return pd.concat([priceInfo.drop(column,axis=1),column_values],axis=1)

priceInfo = Dummify('DOSAGEFORMNAME')
priceInfo = Dummify('ROUTENAME')
priceInfo = Dummify('LABELERNAME')
priceInfo = Dummify('PHARM_CLASSES')
priceInfo = Dummify('DEASCHEDULE')



In [113]:
from sklearn import linear_model
ols = linear_model.LinearRegression()
y = priceInfo["NADAC_Per_Unit"]
x = priceInfo.ix[:, priceInfo.columns != 'NADAC_Per_Unit']

regression = ols.fit(x, y)


In [119]:
print("R^2: %f" %ols.score(x, y))



R^2: 0.943390


In [118]:
colnames = priceInfo.columns
result = pd.DataFrame(ols.coef_).transpose()
result['intercept'] = ols.intercept_ 

result.columns = colnames.tolist()
result = result.transpose()
result.columns = ['coefficient']
result

,coefficient
STARTMARKETINGDATE,0.000052
NADAC_Per_Unit,-48.558194
"DOSAGEFORMNAME__AEROSOL, FOAM",-43.006134
"DOSAGEFORMNAME__AEROSOL, METERED",-50.756912
"DOSAGEFORMNAME__AEROSOL, POWDER",-51.405388
"DOSAGEFORMNAME__AEROSOL, SPRAY",-59.099597
DOSAGEFORMNAME__CAPSULE,-68.571092
"DOSAGEFORMNAME__CAPSULE, COATED PELLETS",-57.996048
"DOSAGEFORMNAME__CAPSULE, COATED, EXTENDED RELEASE",-62.416191
"DOSAGEFORMNAME__CAPSULE, DELAYED RELEASE",-59.972541
